In [3]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('./models')
sys.path.append('./helpers')
import json
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from rank_bm25 import BM25Okapi
from pymongo import MongoClient
import psycopg2

from models.bm25_base import BM25_json
from models.desm import DESM_json


from pydantic import BaseModel
from typing import Dict

class FeedbackItem(BaseModel):
    rank: int
    doc_id: int
    score: float
    userScore: int
    query: str
    algorithm: str
    session_id: int

class Feedback(BaseModel):
    __root__: Dict[str, FeedbackItem]


        
        


# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="SearchEngine",
    user="postgres",
    password="2580"
)

# Create a cursor object
cursor = conn.cursor()


origins = [
    "http://localhost.tiangolo.com",
    "https://localhost.tiangolo.com",
    "http://localhost",
    "http://localhost:8080",
]


app = FastAPI()

# Add the following lines to enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)



@app.get("/search")
async def search(query: str,algorithm: str):
    
    user_input = query
    

    if algorithm == 'BM25':
        out_data, raw_scores = BM25_json(user_input)
        json_data = json.dumps(out_data)

        return json_data
    
    elif algorithm == 'DESM':
        out_data, raw_scores = DESM_json(user_input)
        json_data = json.dumps(out_data)
        
        return json_data

@app.post("/feedback")
async def send_feedback(feedback: Feedback):
    # Your feedback handling logic here
    print(feedback)

    # Extract the feedback data
    items = feedback.__root__
    for key, item in items.items():
        session_id = item.session_id
        query = item.query
        algorithm = item.algorithm
        userScore = item.userScore
        raw_score = item.score

        # Insert the feedback data into the session table
        cursor.execute("""
            INSERT INTO session (number, query, algorithm, userScore, raw_score)
            VALUES (%s, %s, %s, %s, %s)
        """, (session_id, query, algorithm, [userScore], [raw_score]))

    # Commit the changes and close the cursor
    conn.commit()

    return {}




In [5]:
from fastapi.testclient import TestClient

client = TestClient(app)

ModuleNotFoundError: No module named 'httpx'

In [4]:
feedback_json = """
{
    "1": {
        "rank": 1,
        "doc_id": 356,
        "score": 0.125,
        "userScore": 1,
        "query": "hello London",
        "algorithm": "BM25",
        "session_id": 12
    },
    "2": {
        "rank": 2,
        "doc_id": 480,
        "score": 0.05,
        "userScore": 2,
        "query": "hello London",
        "algorithm": "BM25",
        "session_id": 12
    },
    "3": {
        "rank": 3,
        "doc_id": 999,
        "score": 0.04,
        "userScore": 0,
        "query": "hello London",
        "algorithm": "BM25",
        "session_id": 12
    }
}
"""

# Test the endpoint
response = client.post("/feedback", json=json.loads(feedback_json))

print(response.status_code)
print(response.json())


NameError: name 'client' is not defined

In [1]:
import json

In [2]:
feedback_json = """
{
    "1": {
        "rank": "1000",
        "doc_id": "356",
        "score": "0.125",
        "userScore": "1",
        "query": "hello London",
        "algorithm": "BM25",
        "session_id": "126"
    }

} 
"""

In [3]:
json=json.loads(feedback_json)

In [6]:
dict(json).keys()


dict_keys(['1'])

In [3]:
from sklearn.metrics import ndcg_score
import numpy as np

In [48]:
true_relevance = np.array([0, 1, 0, 2, 1]).reshape(1,-1)
scores = np.array([.1, .2, .3, 4, 70]).reshape(1,-1)

In [49]:
scores.shape, true_relevance.shape

((1, 5), (1, 5))

In [50]:
ndcg_score(true_relevance, scores)

0.8599797111848091

In [43]:
true_relevance = np.asarray([[10, 0, 0, 1, 5]])

In [29]:
scores = np.asarray([[.1, .2, .3, 4, 70]])

In [30]:
scores.shape

(1, 5)

In [31]:
ndcg_score(true_relevance, scores)

0.6956940443813076